In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


We need to process these steps just once and then for all the models we can simply just use the directory after mounting to drive

These cells are not run as at somepoint during the project these folders were already created

Loading data for the bloodmnist dataset

In [2]:
!mkdir -p "/content/drive/MyDrive/MedMNIST"
import os
os.chdir("/content/drive/MyDrive/MedMNIST")
!apt-get install -y git
!git clone https://github.com/MedMNIST/MedMNIST.git

#here we are assuming that blood_dataset folder is created
!cp '/content/drive/MyDrive/MedMNIST/MedMNIST/examples/dataset_without_pytorch.py' "/content/drive/MyDrive/blood_dataset"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=00663dae7c25cf6ebfb10fc373afdbf3ab841ad8d55da48157b2d521e99e0da8
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [10]:
import dataset_without_pytorch
from dataset_without_pytorch import get_loader

In [ ]:
import os
import numpy as np
from tqdm import tqdm
import medmnist
from medmnist import INFO, Evaluator

print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

data_flag = 'bloodmnist'
download = True

BATCH_SIZE = 128

class_map = {
    '0': 'basophil',
    '1': 'eosinophil',
    '2': 'erythroblast',
    '3': 'immature_granulocytes',
    '4': 'lymphocyte',
    '5': 'monocyte',
    '6': 'neutrophil',
    '7': 'platelet'
}

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(dataset_without_pytorch, info['python_class'])

BATCH_SIZE = 128
# load the data
train_dataset = DataClass(split='train', download=download)
test_dataset = DataClass(split='test', download=download)
validation_dataset = DataClass(split='val', download=download)

train_loader = get_loader(dataset=train_dataset, batch_size=BATCH_SIZE)
test_loader = get_loader(dataset=test_dataset, batch_size=BATCH_SIZE)
val_loader = get_loader(dataset=validation_dataset, batch_size=BATCH_SIZE)

# Path to the main 'blood_dataset' folder
output_folder = '/content/drive/MyDrive/blood_dataset'


def save_images_with_labels(dataset, data_split, output_folder):
    for i, (image, label) in tqdm(enumerate(dataset), total=len(dataset), desc=f"Saving {data_split} images"):
        class_label = label[0] if isinstance(label, np.ndarray) else label
        class_name = class_map[str(class_label)]  # Get the class name from the class_map
        class_folder = os.path.join(output_folder, data_split, class_name)
        os.makedirs(class_folder, exist_ok=True)

        filename = f"{data_split}_{class_name}_{i:04d}.png"  # Assuming you want images in PNG format
        output_path = os.path.join(class_folder, filename)

        # Save the image
        image.save(output_path)

# Save images from the 'train' dataset into the 'train' subfolder
save_images_with_labels(train_dataset, 'train', output_folder)

# Save images from the 'test' dataset into the 'test' subfolder
save_images_with_labels(test_dataset, 'test', output_folder)

# Save images from the 'validation' dataset into the 'validation' subfolder
save_images_with_labels(validation_dataset, 'validation', output_folder)


Load dataset for breast mnist

In [ ]:
#create a folder manually called breast_mnist
import os
os.chdir("/content/drive/MyDrive/MedMNIST/breast_mnist")

In [ ]:
!apt-get install -y git
!git clone https://github.com/MedMNIST/MedMNIST.git

In [ ]:
!cp /content/drive/MyDrive/MedMNIST/breast_mnist/MedMNIST/examples/dataset_without_pytorch.py "/content/drive/MyDrive/MedMNIST/breast_mnist"


In [ ]:
import os
os.chdir("/content/drive/MyDrive/MedMNIST/breast_mnist")

In [ ]:
!pip install medmnist

In [ ]:
import dataset_without_pytorch
from dataset_without_pytorch import get_loader

In [ ]:
from tqdm import tqdm
import numpy as np

import medmnist
from medmnist import INFO, Evaluator

print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

data_flag = 'breastmnist'

download = True


BATCH_SIZE = 128


info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(dataset_without_pytorch, info['python_class'])

BATCH_SIZE = 128
# load the data
train_dataset = DataClass(split='train', download=download)
test_dataset = DataClass(split='test', download=download)
validation_dataset = DataClass(split='val', download=download)

train_loader = get_loader(dataset=train_dataset, batch_size=BATCH_SIZE)
test_loader = get_loader(dataset=test_dataset, batch_size=BATCH_SIZE)
val_loader = get_loader(dataset=validation_dataset, batch_size=BATCH_SIZE)

train_images=train_dataset.imgs
train_labels=train_dataset.labels
val_images=validation_dataset.imgs
val_labels=validation_dataset.labels
test_images=test_dataset.imgs
test_labels=test_dataset.labels

import os
import cv2
import shutil

# Function to move images to their respective subfolders based on the dataset (train, test, val)
def move_images_to_subfolders(images, labels, dataset_name, save_folder):
    dataset_folder = os.path.join(save_folder, dataset_name)
    os.makedirs(dataset_folder, exist_ok=True)

    for i, (image, label) in enumerate(zip(images, labels)):
        # Create the label subfolder if it doesn't exist in the dataset folder
        label_folder = os.path.join(dataset_folder, str(label))
        os.makedirs(label_folder, exist_ok=True)

        # Convert the image to RGB if it's not already
        if image.shape[-1] == 1:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

        # Save the image in the corresponding label subfolder
        image_path = os.path.join(label_folder, f"image_{i}.png")
        cv2.imwrite(image_path, image)

# Assuming `breast_mnist` is the dataset folder
dataset_folder = 'breast_mnist'

# Categorize and move images and labels for the train dataset
move_images_to_subfolders(train_images, train_labels, 'train', dataset_folder)

# Categorize and move images and labels for the test dataset
move_images_to_subfolders(test_images, test_labels, 'test', dataset_folder)

# Categorize and move images and labels for the validation dataset
move_images_to_subfolders(val_images, val_labels, 'val', dataset_folder)
